In [21]:
from pathlib import Path

current_repo_path = Path().parent.resolve()

In [22]:
TARGET_REPO = "devQ_testData_PythonProject"
repo_path = current_repo_path.parent / TARGET_REPO

In [23]:
from developerscope.analyzer import get_merge_commits_map

merge_commits_map, author_mapping = get_merge_commits_map(repo_path)
author_mapping

defaultdict(set,
            {'iatskovskiivv': {('IatskovskiiVV@alfastrah.ru',
               'Яцковский Владимир'),
              ('iatskovskiivv@alfastrah.ru', 'Яцковский Владимир Валерьевич')},
             'isaninav': {('isaninav@alfastrah.ru', 'Isanin Anton'),
              ('isaninav@alfastrah.ru', 'antisanin'),
              ('isaninav@alfastrah.ru', 'Исанин Антон Викторович')},
             'panfilovva': {('PanfilovVA@alfastrah.ru', 'PanfilovVA'),
              ('panfilovva@alfastrah.ru', 'Панфилов Вадим Алексеевич')},
             'vasilevar': {('vasilevar@alfastrah.ru',
               'Васильев Артём Романович')},
             'avasilev1992': {('avasilev1992@gmail.com', 'Artem Vasilev')},
             'vasilevartem': {('53534684+VasilevArtem@users.noreply.github.com',
               'Artem Vasilev')}})

In [24]:
from developerscope.analyzer import get_all_branches

get_all_branches(repo_path)

[<git.Head "refs/heads/develop">,
 <git.Head "refs/heads/hackaton">,
 <git.Head "refs/heads/master">]

In [25]:
from collections import defaultdict
from pydriller import Repository
from developerscope.analyzer import extract_username

merge_commts_map = defaultdict(list)
author_mapping = defaultdict(set)

repo = Repository(str(repo_path), only_in_branch='master')
count = 0
for commit in repo.traverse_commits():
    count += 1
    if not commit.msg.startswith('Merge'):
        continue

    # print(commit.msg.__repr__())
    username = extract_username(commit.author.email)

    author_mapping[username].add((commit.author.email, commit.author.name))

    merge_commts_map[username].append(commit.hash)
count

483

In [7]:
from collections import defaultdict


merge_commits_difference = defaultdict(list)

In [20]:
merge_commits_map['vasilevar']

['7f90432badcc1ac4c926d3fcbf5120eded4ed91f',
 '3ffacb0cd56b85056f31a8edaf38eeb857944d61',
 'd4fe6ed027392f36304c1e1da7f05bce92010da8',
 '37c66b462bfa87ba4c0f2e96d605f54df1656ecb',
 '9f6c2e59567339e3ec67839ce2257a19c1b34090',
 'e7e6b8880c5f009b5bf455a19be5cef1fc91c39e',
 'a6746cc68380dba972e789db0dd6668c861567fe',
 '02fdc2f8086a1cb90d96c84a16647037a7ab339a',
 'ec06dc870e02ab0ff066095e75de322a927d54f8',
 '9837f02db3fce88d2990e5d0859175a5d45c3d11',
 'cc28d15f6e4a59b06b6a301f764fbb4f7fe7abb4',
 '39aecb4ec9ae1439a2e36efb6d1ce2b1aba38237',
 'ff56e463764905c6c041c9ce770e80d7409b251f',
 'c64c0b74e241376f33dfaf0c831ce697f6dd1f1c',
 '6ad3ae301b9386fa2ae97efe3591ebbb8e950811',
 'a3358b852fb53524808b83ce0a05664cebf3faf4',
 '6e532bb2156ad09650167a880ac32b5c97ef5822',
 '285410f4bc02cc82d8e8478a39702ede88f9e71c',
 '7c7a0e9e4231831d4f50d72c374a325d077082da',
 '29be31f923b36386abde9f837ec61598ab33da20',
 '44bebdc13f5688c324fde297d07bfed4287fc8e5',
 '493e23c72795f1496532055f02b4fe1f01999a63',
 '4297e785

In [13]:
from developerscope.analyzer import get_difference

import git
git_repo = git.Repo(str(repo_path))

current_folder_name = current_repo_path.name
output_base = current_repo_path / "current_repo"

for author, merge_requests in merge_commits_map.items():
    for merge_request in merge_requests:
        merge_commit = git_repo.commit(merge_request)

        # Prepare directory path
        output_dir = output_base / author
        output_dir.mkdir(parents=True, exist_ok=True)

        # Prepare file path
        output_file = output_dir / f"{merge_commit.hexsha}.txt"

        # Write to file
        with open(output_file, "w", encoding="utf-8") as f:
            diff = get_difference(merge_commit)
            f.write(diff)


In [11]:
print(diff)

==== File: .gitlab-ci.yml ====
@@ -1,115 +0,0 @@
-
-variables:
-  SITE_ROOT: /var/www/acl.vesta.ru
-  
-stages:
-  - lint
-  - deploy
-  - rollback
-  - cleanup
-
-workflow:
-  rules:
-    - if: $CI_COMMIT_TAG =~ /^prod_/
-    - if: $CI_COMMIT_TAG =~ /^test_/
-
-lint_code:
-  image: nx-docker-proxy.alfastrah.ru/pipelinecomponents/pylint:amd64-3ea6e68
-  stage: lint
-  script:
-    - ls -las
-    - pylint --exit-zero --load-plugins=pylint_gitlab --output-format=gitlab-codeclimate:codeclimate.json **/*.py
-  artifacts:
-    reports:
-      codequality: codeclimate.json
-    when: always
-  allow_failure: true
-
-  tags: [ k8s-alfastrah]
-
-.deploy_web:
-  stage: deploy
-  script:
-    - backup_archive=/var/tmp/acl-backup$(date "+%Y%m%d$H%M%S")
-    - echo "BACKUP_ARCHIVE=$backup_archive" >> deploy_web.env
-    - tar -czf $backup_archive.tar.gz $SITE_ROOT
-    - ls -las $CI_PROJECT_DIR
-    - cp -r $CI_PROJECT_DIR/* $SITE_ROOT
-    - if grep -q '^COMMIT_TAG=' $SITE_ROOT/.env; then
-      

In [23]:
merge_commit_hash: str = merge_request

merge_commit = git_repo.commit(merge_commit_hash)
first_parent = git_repo.commit(merge_commit_hash + '^1')


In [27]:
# Collect all files in the commit
file_chunks = []

for blob in merge_commit.tree.traverse():
    if blob.type == 'blob':  # it's a file
        file_path = blob.path
        file_content = blob.data_stream.read().decode('utf-8', errors='replace')
        print(file_path)
        formatted = f"### FILE: `{file_path}`\n\n```\n{file_content}\n```\n"
        file_chunks.append(formatted)

# Join all file contents into a single string
final_string = "\n\n".join(file_chunks)

# Save to a markdown file
output_file = Path("prompt.md")
output_file.write_text(final_string, encoding='utf-8')


.gitignore
manage.py
requirements.txt
accesslist/__init__.py
accesslist/admin.py
accesslist/apps.py
accesslist/auth.py
accesslist/forms.py
accesslist/models.py
accesslist/signals.py
accesslist/tests.py
accesslist/urls.py
accesslist/views.py
acladmin/__init__.py
acladmin/asgi.py
acladmin/celery.py
acladmin/context_processors.py
acladmin/settings.py
acladmin/tasks.py
acladmin/urls.py
acladmin/views.py
acladmin/wsgi.py
acladmin/yasg.py
ownerlist/__init__.py
ownerlist/admin.py
ownerlist/apps.py
ownerlist/docx_hotfix.py
ownerlist/forms.py
ownerlist/models.py
ownerlist/tests.py
ownerlist/urls.py
ownerlist/utils.py
ownerlist/views.py
panel/__init__.py
panel/admin.py
panel/apps.py
panel/forms.py
panel/models.py
panel/tests.py
panel/urls.py
panel/views.py
teams/__init__.py
teams/admin.py
teams/apps.py
teams/models.py
teams/tests.py
teams/urls.py
teams/views.py
templates/403.html
templates/404.html
templates/500.html
templates/ACL.docx
templates/base.html
tests/__init__.py
accesslist/management/

3745112

In [1]:
from openai import AsyncOpenAI
import asyncio

client = AsyncOpenAI()


async def main() -> None:
    response = await client.responses.create(
        model="gpt-4o", input="Explain disestablishmentarianism to a smart five year old."
    )
    print(response.output_text)


asyncio.run(main())


RuntimeError: asyncio.run() cannot be called from a running event loop